In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_45_topics/lda_model_45_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 24: 0.176*"women" + 0.058*"south" + 0.037*"basketball" + 0.035*"wnba" + 0.019*"connecticut" + 0.016*"gamecocks" + 0.012*"auriemma" + 0.012*"article" + 0.011*"player" + 0.010*"moore"
Topic 19: 0.039*"season" + 0.029*"guard" + 0.018*"rebounds" + 0.015*"forward" + 0.014*"assists" + 0.014*"scoring" + 0.014*"freshman" + 0.013*"player" + 0.013*"team" + 0.012*"shooting"
Topic 30: 0.025*"team" + 0.019*"girls" + 0.019*"school" + 0.014*"gcu" + 0.012*"college" + 0.012*"senior" + 0.012*"named" + 0.012*"division" + 0.011*"award" + 0.009*"player"
Topic 28: 0.130*"gophers" + 0.102*"minnesota" + 0.032*"ten" + 0.023*"staff" + 0.019*"dakota" + 0.016*"season" + 0.014*"team" + 0.012*"johnson" + 0.011*"north_dakota" + 0.011*"startribune.com"
Topic 1: 0.056*"nba" + 0.025*"draft" + 0.022*"basketball" + 0.019*"pick" + 0.018*"league" + 0.013*"player" + 0.013*"celtics" + 0.013*"lakers" + 0.012*"star" + 0.010*"professional"
Topic 20: 0.034*"sec" + 0.023*"season" + 0.022*"top" + 0.021*"win" + 0.019*

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import json
import pandas as pd

# Assuming your JSON file contains a list of lists, where each inner list is a tokenized document
# Replace '/content/drive/MyDrive/path/to/your/texts.json' with the actual path to your JSON file
texts_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json'

with open(texts_file_path, 'r') as f:
    texts = json.load(f)

print(f"Loaded {len(texts)} documents from {texts_file_path}")

Loaded 149649 documents from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json


In [ ]:
# import pandas as pd

# # Load the tokenized data from the Excel file
# try:
#     tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
#     print("Excel file loaded successfully.")
#     # Assuming the original text is in a column named 'text' or similar.
#     # Please adjust the column name if it's different in your Excel file.
#     if 'Main_text_tokenized' in tokenized_data_df.columns:
#         texts = tokenized_data_df['Main_text_tokenized'].tolist()
#         print(f"Extracted {len(texts)} documents.")
#     else:
#         texts = None
#         print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

# except FileNotFoundError:
#     print("Error: The file was not found at the specified path.")
# except Exception as e:
#     print(f"An error occurred while reading the Excel file: {e}")

Excel file loaded successfully.
Extracted 149703 documents.


In [5]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.4325,"family, school, life, time, people, basketball...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.2554,"family, school, life, time, people, basketball...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,43,0.3364,"fans, time, people, sports, day, moment, artic...","[garden, missed, lonely, nights, season, debut..."
3,3,16,0.2316,"team, season, players, play, lot, time, playin...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,31,0.2168,"league, season, baseball, games, professional,...","[robinson, talk, rangers, stanley, cup, finals..."
5,5,19,0.2858,"season, guard, rebounds, forward, assists, sco...","[conference, pacific, coach_mike, gillespie, s..."
6,6,31,0.3528,"league, season, baseball, games, professional,...","[rangers, signed, center, michael, nylander, m..."
7,7,31,0.2631,"league, season, baseball, games, professional,...","[gavitt, elected, hall, modest, player, dartmo..."
8,8,44,0.2184,"half, lead, minutes, shot, left, rebounds, pla...","[mythical, element, senior_guard, seimone, aug..."
9,9,4,0.3633,"family, school, life, time, people, basketball...","[ncaa_women, jim, izard, diana, vines, remembe..."


In [6]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.4325,"family, school, life, time, people, basketball...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,4,0.2554,"family, school, life, time, people, basketball...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,43,0.3364,"fans, time, people, sports, day, moment, artic...","[garden, missed, lonely, nights, season, debut..."
3,3,16,0.2316,"team, season, players, play, lot, time, playin...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,31,0.2168,"league, season, baseball, games, professional,...","[robinson, talk, rangers, stanley, cup, finals..."


In [7]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[3064, 48272, 47626, 60670, 96487, 50073, 79657, 37592, 65983, 72012, 3062, 41663, 39259, 48437, 75518, 74180, 21372, 36857, 17304, 82691]

Topic 1:
[60308, 35678, 46827, 123733, 85984, 50512, 8388, 24210, 101871, 46828, 50513, 50511, 103313, 39842, 80593, 148177, 27269, 13271, 92178, 134431]

Topic 2:
[34130, 52659, 77138, 67265, 93194, 129597, 5007, 93468, 129598, 55378, 129599, 129461, 93060, 15655, 19921, 124883, 95049, 94998, 28895, 131184]

Topic 3:
[131802, 77022, 43636, 44086, 11292, 70761, 8994, 120101, 19333, 53822, 8995, 94717, 62891, 131787, 43348, 30644, 17582, 70162, 5241, 50074]

Topic 4:
[87936, 63005, 123383, 140488, 131126, 85340, 19603, 75224, 42581, 133552, 1977, 126779, 58259, 141363, 136513, 11603, 79638, 137579, 76784, 3211]

Topic 5:
[25870, 33888, 24077, 14542, 115822, 100438, 28071, 67515, 77012, 16414, 57162, 14534, 57083, 121819, 78292, 49104, 91429, 43823, 14537, 43182]

Topic 6:

In [8]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,2596
1,2903
2,4675
3,2907
4,5675
5,6183
6,352
7,7077
8,2902


In [9]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,3064,0,0.9279,"athletes, university, school, sports, schools,...","[colleges, graduation, rate, division, student..."
1,48272,0,0.9035,"athletes, university, school, sports, schools,...","[university, federal, graduation_rates, athlet..."
2,47626,0,0.8509,"athletes, university, school, sports, schools,...","[graduation_rates, division, basketball, baseb..."
3,60670,0,0.8439,"athletes, university, school, sports, schools,...","[ncaa, academic, reforms, refined, eligibility..."
4,96487,0,0.8349,"athletes, university, school, sports, schools,...","[football_program, improved, academic, progres..."
...,...,...,...,...,...
95,6516,0,0.6017,"athletes, university, school, sports, schools,...","[rice, football, field, owls, wisest, graduati..."
96,22708,0,0.6007,"athletes, university, school, sports, schools,...","[school, math, college, degree, political, sci..."
97,133433,0,0.6004,"athletes, university, school, sports, schools,...","[fifty, title_ix, passed, colleges, universiti..."
98,4800,0,0.5993,"athletes, university, school, sports, schools,...","[n.c.a.a, division, board, directors, approved..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,60308,1,0.7271,"nba, draft, basketball, pick, league, player, ...","[timberwolves, entered, thursday, night, draft..."
101,35678,1,0.667,"nba, draft, basketball, pick, league, player, ...","[bucks, suffered, disappointing, season, playo..."
102,46827,1,0.6477,"nba, draft, basketball, pick, league, player, ...","[portland, beat, odds, tuesday, won, nba_draft..."
103,123733,1,0.6271,"nba, draft, basketball, pick, league, player, ...","[ers, lucky, miss, prospect, hoped, snag, pick..."
104,85984,1,0.5974,"nba, draft, basketball, pick, league, player, ...","[happy, coincidence, tom, benson, newly, minte..."
...,...,...,...,...,...
195,48351,1,0.4672,"nba, draft, basketball, pick, league, player, ...","[listed, mock_drafts, ers, pick, uncertainty, ..."
196,45809,1,0.4671,"nba, draft, basketball, pick, league, player, ...","[timberwolves, played, wednesday, play, night,..."
197,63329,1,0.4667,"nba, draft, basketball, pick, league, player, ...","[plan, forty, hours, blake, griffin, stroll, p..."
198,5340,1,0.4665,"nba, draft, basketball, pick, league, player, ...","[nba, finals, lakers, nba, finals, bob, keisse..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3734,69272,42,0.3156,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[top_ranked, women, gymnastics, team, won, aft..."
3735,96054,42,0.2757,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[women, gymnastics, coach, spini, coaching, ru..."
3736,14760,42,0.2636,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[freshman, kate, richardson, accomplished, goa..."
3737,11566,42,0.2464,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[balance, start, left, deal, women, gymnastics..."
3738,147801,42,0.243,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[sooners, women, gymnastics, team, sooners, su..."
3739,147701,42,0.2287,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[gymnastics, historically, boasts, olympians, ..."
3740,7340,42,0.2201,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[clay, strother, minnesota, repeated, champion..."
3741,120093,42,0.2189,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[gymnastics, stakes, ncaa, semifinals, teams, ..."
3742,5014,42,0.208,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[jamie, dantzscher, score, uneven, bars, final..."
3743,3576,42,0.2058,"gonzaga, san_diego, gymnastics, bulldogs, usf,...","[college_basketball, dan, dickau, transferred,..."



--- Topic 43 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3759,33969,43,0.6818,"fans, time, people, sports, day, moment, artic...","[ncaa_tournament, diary, mad, fan, day, road, ..."
3760,53099,43,0.6406,"fans, time, people, sports, day, moment, artic...","[wondering, classes, empty, wearing, gold, blu..."
3761,76949,43,0.6341,"fans, time, people, sports, day, moment, artic...","[week, reader, wrote, concerned, succumbed, di..."
3762,62398,43,0.6335,"fans, time, people, sports, day, moment, artic...","[lot, spam, usual, republic, spam, filter, blo..."
3763,14041,43,0.6237,"fans, time, people, sports, day, moment, artic...","[bombs, fall, baghdad, grisham, nightstand, mi..."
...,...,...,...,...,...
3854,47855,43,0.4872,"fans, time, people, sports, day, moment, artic...","[guys, wearing, blue, sweatshirts, letter, fro..."
3855,53907,43,0.4857,"fans, time, people, sports, day, moment, artic...","[feeling, left, office, follow, college_basket..."
3856,112951,43,0.4852,"fans, time, people, sports, day, moment, artic...","[enter, couch, slouch, wishes, war, world, jim..."
3857,12636,43,0.4846,"fans, time, people, sports, day, moment, artic...","[peek, window, beautiful, morning, watch, peop..."



--- Topic 44 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
3859,105853,44,0.8569,"half, lead, minutes, shot, left, rebounds, pla...","[grayson, allen, including, pointers, seed, fe..."
3860,46221,44,0.8142,"half, lead, minutes, shot, left, rebounds, pla...","[ncaa_final, indiana, sacramento, arron, affla..."
3861,24351,44,0.8044,"half, lead, minutes, shot, left, rebounds, pla...","[deron, williams, released, ball, fourth, poin..."
3862,59896,44,0.7981,"half, lead, minutes, shot, left, rebounds, pla...","[complemented, stifling, defense, balanced, of..."
3863,5126,44,0.7976,"half, lead, minutes, shot, left, rebounds, pla...","[madison, wis, personal, fouls, nick, collison..."
...,...,...,...,...,...
3954,14254,44,0.7046,"half, lead, minutes, shot, left, rebounds, pla...","[cushion, leading, minutes, play, seeded, foun..."
3955,128111,44,0.7041,"half, lead, minutes, shot, left, rebounds, pla...","[halftime, deficit, win, requires, improvement..."
3956,36903,44,0.7032,"half, lead, minutes, shot, left, rebounds, pla...","[special, section, ncaa_tournament, elite, reg..."
3957,82867,44,0.7031,"half, lead, minutes, shot, left, rebounds, pla...","[saturday, night, national, semifinal, superdo..."


In [10]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_45_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_45_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**